# Pandas Cheat Sheet — Quick Reference

This notebook provides concise, runnable pandas examples and a few practice problems with solutions.

In [1]:
import pandas as pd
import numpy as np
print('pandas', pd.__version__, 'numpy', np.__version__)

pandas 2.3.3 numpy 2.1.0


## Create DataFrame
Common constructors from dict, list of dicts, and numpy arrays.

In [2]:
df = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie', 'Diane'],
    'age': [25, 32, 37, 29],
    'city': ['NY', 'LA', 'NY', 'SF']
})
df

,name,age,city
0,Alice,25,NY
1,Bob,32,LA
2,Charlie,37,NY
3,Diane,29,SF


## Inspecting data
Use `head()`, `info()`, `describe()`, attributes like `shape`, `columns`, `dtypes`.

In [7]:
df.head()
df.info()
df.describe(include='all')
df.shape, df.columns, df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    4 non-null      object
 1   age     4 non-null      int64 
 2   city    4 non-null      object
dtypes: int64(1), object(2)
memory usage: 228.0+ bytes


((4, 3),
 Index(['name', 'age', 'city'], dtype='object'),
 name    object
 age      int64
 city    object
 dtype: object)

## Selection & Indexing
- Column selection: `df['col']` or `df.col`
- Row selection: `df.loc[label]`, `df.iloc[pos]`
- Boolean selection and chaining examples below.

In [12]:
# column and row selection examples
df['age']
df.loc[0]          # first row by index label
df.iloc[1:3]       # slice by position
df[df['age'] > 30] # boolean filter

,name,age,city
1,Bob,32,LA
2,Charlie,37,NY


## Common operations
Aggregation, `groupby`, `merge`, `pivot_table`, `apply`, handling missing values.

In [13]:
orders = pd.DataFrame([
    {'order_id': 1, 'customer': 'Alice', 'product': 'A', 'qty': 2, 'price': 9.99, 'region': 'East', 'date': '2021-01-05'},
    {'order_id': 2, 'customer': 'Bob', 'product': 'B', 'qty': 1, 'price': 19.99, 'region': 'West', 'date': '2021-02-10'},
    {'order_id': 3, 'customer': 'Alice', 'product': 'A', 'qty': 5, 'price': 9.99, 'region': 'East', 'date': '2021-02-17'},
    {'order_id': 4, 'customer': 'Diane', 'product': 'C', 'qty': 3, 'price': 4.5, 'region': np.nan, 'date': '2021-03-01'},
    {'order_id': 5, 'customer': 'Charlie', 'product': 'B', 'qty': 2, 'price': 19.99, 'region': 'NY', 'date': '2021-03-05'}
])
orders['total'] = orders['qty'] * orders['price']
orders['date'] = pd.to_datetime(orders['date'])
orders

,order_id,customer,product,qty,price,region,date,total
0,1,Alice,A,2,9.99,East,2021-01-05,19.98
1,2,Bob,B,1,19.99,West,2021-02-10,19.99
2,3,Alice,A,5,9.99,East,2021-02-17,49.95
3,4,Diane,C,3,4.50,NaN,2021-03-01,13.50
4,5,Charlie,B,2,19.99,NY,2021-03-05,39.98


In [36]:
# groupby aggregation example
orders.groupby('customer')['total'].sum().reset_index(name='spent')
# pivot table example
pd.pivot_table(orders, index='customer', columns='product', values='qty', aggfunc='sum', fill_value=0)
# merge example
cust = pd.DataFrame({'customer': ['Alice','Bob','Eve'], 'vip': [True, False, False]})
orders.merge(cust, on='customer', how='left')

,order_id,customer,product,qty,price,region,date,total,vip
0,1,Alice,A,2,9.99,East,2021-01-05,19.98,True
1,2,Bob,B,1,19.99,West,2021-02-10,19.99,False
2,3,Alice,A,5,9.99,East,2021-02-17,49.95,True
3,4,Diane,C,3,4.50,NaN,2021-03-01,13.50,NaN
4,5,Charlie,B,2,19.99,NY,2021-03-05,39.98,NaN


## Missing data
Use `isna()`, `dropna()`, `fillna()`, or impute with stats.

In [39]:
orders.isna().sum()
orders['region'].fillna('Unknown', inplace=False)
orders.dropna(subset=['region'])

,order_id,customer,product,qty,price,region,date,total
0,1,Alice,A,2,9.99,East,2021-01-05,19.98
1,2,Bob,B,1,19.99,West,2021-02-10,19.99
2,3,Alice,A,5,9.99,East,2021-02-17,49.95
4,5,Charlie,B,2,19.99,NY,2021-03-05,39.98


---
## Practice Problems
Below are small problems using the `orders` DataFrame above. Each problem is followed by an executable solution cell.

**Problem 1:** Select orders made by 'Alice' and compute total quantity she ordered.

In [42]:
# Solution 1
alice_orders = orders[orders['customer'] == 'Alice']
alice_qty = alice_orders['qty'].sum()
alice_qty

np.int64(7)

**Problem 2:** Add a new column `unit_price` equal to `price` and a `discounted` column which is `total` with a 10% discount applied for orders with `qty` >= 3.

In [44]:
# Solution 2
orders2 = orders.copy()
orders2['unit_price'] = orders2['price']
orders2['After_discounte'] = orders2['total'] * np.where(orders2['qty'] >= 3, 0.9, 1.0)
orders2[['order_id','qty','unit_price','total','After_discounte']]

,order_id,qty,unit_price,total,After_discounte
0,1,2,9.99,19.98,19.980
1,2,1,19.99,19.99,19.990
2,3,5,9.99,49.95,44.955
3,4,3,4.50,13.50,12.150
4,5,2,19.99,39.98,39.980


**Problem 3:** Compute total revenue per `region` (treat missing `region` as 'Unknown').

In [45]:
# Solution 3
orders3 = orders.copy()
orders3['region'] = orders3['region'].fillna('Unknown')
revenue_by_region = orders3.groupby('region')['total'].sum().reset_index(name='revenue')
revenue_by_region

,region,revenue
0,East,69.93
1,NY,39.98
2,Unknown,13.50
3,West,19.99


**Problem 4:** Create a pivot table showing total `qty` per `customer` per `product`.

In [46]:
# Solution 4
pivot = pd.pivot_table(orders, index='customer', columns='product', values='qty', aggfunc='sum', fill_value=0)
pivot

product,A,B,C
customer,,,
Alice,7,0,0
Bob,0,1,0
Charlie,0,2,0
Diane,0,0,3
